In [303]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [304]:
import pandas as pd

df = pd.read_csv('permisos_construccion.csv')

C:\Users\germa\AppData\Local\Temp\ipykernel_14108\3263701215.py:3: DtypeWarning: Columns (22,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('permisos_construccion.csv')


### Tomar una decisión respecto a los datos faltantes, duplicados, inconsistencias y cualquier inconveniente que encuentren con los datos.

+ `Street Name` tiene distintos tipos de usos de mayúsculas y minúsculas. Se va a pasar todo a mayúsculas en la primera letra y el resto en minúsculas.
+ `Street Suffix` tiene valores nulos y valores que no son nulos. Se va a unir esta columna con `Street Name` y se va a eliminar `Street Name Suffix`.

In [305]:
# `Street Name` tiene distintos tipos de usos de mayúsculas y minúsculas. Se va a pasar todo a mayúsculas en la primera letra y el resto en minúsculas.
df['Street Name'] = df['Street Name'].str.title() + ' ' + df['Street Suffix'].fillna('')
# 'Street Suffix' tiene valores nulos. Se va a eliminar la columna y se va a unir con Street Name.
df.drop(columns=['Street Suffix'], inplace=True)

+ `Supervisor District` tiene algunas inconsistencias en los valores:

In [306]:
#'Supervisor District' null values?
null_sd = df['Supervisor District'].isnull().sum()
print(f'Hay {null_sd} valores nulos en la columna Supervisor District')
#'Supervisor District' valores repetidos? Count
count_sd = df['Supervisor District'].value_counts()
print(f'Los valores de Supervisor District son:\n{count_sd}')

Hay 1719 valores nulos en la columna Supervisor District
Los valores de Supervisor District son:
Supervisor District
3.0       21691
8.0       20112
2.0       19216
6.0       18650
5.0       14297
9.0       12303
7.0       10860
1.0        9832
10.0       9129
4.0        7223
3.0        6958
8.0        6649
2.0        6268
6.0        6147
11.0       5192
5.0        4748
9.0        4059
7.0        3505
1.0        3207
10.0       3024
4.0        2369
11.0       1749
quince        1
veinte        1
diez          1
Name: count, dtype: int64


+ Para `Supervisor District` pasamos los números escritos en minúsculas a strings con dígitos.

In [307]:
# "Supervisor District" tiene números redactados. Se van a cambiar por numeración para facilitar la lectura pero como valor de string.
number_mapping = {
"diez":"10.0",
"quince":"15.0",
"veinte":"20.0",
}

df['Supervisor District'] = df['Supervisor District'].replace(number_mapping)
df['Supervisor District'] = df['Supervisor District'].astype(str)

+ En `TIDF Compliance` se elimino la columna debido a que unicamente tiene 2 valores no nulos.
+ Se hizo algo similar para `Plansets`, rellenando con 0 los que estaban vacíos.
+ En `Plansets` se removieron las filas con valores mayor a 100, se eligio este valor arbitrario para tener un margen amplio

In [308]:
#'TIDF Compliance' tiene unicamente 2 valores no nulos. Se va a eliminar la columna.
df.drop('TIDF Compliance', inplace=True, axis=1)
df['Plansets'] = df['Plansets'].fillna(0)
for x in df.index:
  if df.loc[x, 'Plansets'] > 100:
    df.drop(x, inplace = True)

+ Para `Existing Construction Type`, `Existing Construction Type Description`, `Proposed Construction Type`, `Proposed Construction Type Description` decidimos borrar las descripciones que son incoherentes, como cuando la descripcion reitera el valor sin proveer mayor descripcion. Por ejemplo, cuando el valor de `Existing Construction Type` es `3` y el `Existing Construction Type Description` es `constr type 3`

In [309]:
print(df[['Existing Construction Type', 'Existing Construction Type Description']].head(10))

   Existing Construction Type Existing Construction Type Description
0                         NaN                                    NaN
1                         5.0                         wood frame (5)
2                         5.0                         wood frame (5)
3                         5.0                         wood frame (5)
4                         5.0                         wood frame (5)
5                         5.0                         wood frame (5)
6                         NaN                                    NaN
7                         NaN                                    NaN
8                         3.0                          constr type 3
9                         3.0                          constr type 3


In [310]:
df['Existing Construction Type Description'] = df['Existing Construction Type Description'].apply(
    lambda desc: np.nan if desc in [' constr type 3 ', ' constr type 1 ', 'constr type 1', 'constr type 2', 'constr type 3', 'constr type 4', 'constr type 5']
    else 'wood frame' if desc in ['wood frame (5)', ' wood frame (5) ']
    else desc
)

    

+ En `Permit Type Definition`, hay algunos valores que representan lo mismo, pero tiene nombres diferentes o insconcistencias en el typeo, se decide modificar todo a un nombre comun. Por ejemplo 'new construction', hay variantes como ' new construction', 'new construction #'

In [311]:
df['Permit Type Definition'] = df['Permit Type Definition'].apply(
    lambda desc: 'new construction' if desc in ['new construction #', ' new construction ']
    else 'additions alterations or repairs' if desc in [' additions alterations or repairs ']
    else 'otc alterations permit' if desc in [' otc alterations permit ', 'otc alterations permit #']
    else desc
)

count_sd = df['Permit Type Definition'].value_counts()
print(f'Los valores de Permit Type Definition son:\n{count_sd}')

Los valores de Permit Type Definition son:
Permit Type Definition
otc alterations permit                 178835
additions alterations or repairs        14663
sign - erect                             2892
new construction wood frame               950
demolitions                               600
wall or painted sign                      511
new construction                          347
grade or quarry or fill or excavate        91
 otc alterations permit                     8
otc alterations permit #                    8
new construction #                          2
 new construction                           1
 additions alterations or repairs           1
Name: count, dtype: int64
Los valores de Permit Type Definition son:
Permit Type Definition
otc alterations permit                 178851
additions alterations or repairs        14664
sign - erect                             2892
new construction wood frame               950
demolitions                               600
wall or painte